In [1]:
# NEED THIS TO SAVE TO BIGGER DRIVE
import os
os.environ['HF_HOME'] = '/datastor1/jiahuikchen/hf_cache'

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from diffusers import StableUnCLIPImg2ImgPipeline
from diffusers.utils import load_image
import torch
from torchvision.transforms import v2

In [2]:
# cutmix/mixup
cutmix = v2.CutMix(num_classes=1)
mixup = v2.MixUp(num_classes=1)
preprocess = v2.Compose([
    v2.PILToTensor(), 
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.ToDtype(torch.float32, scale=True)
])

img_txt_pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float16, 
)
img_txt_pipe = img_txt_pipe.to("cuda:0")

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]